In [ ]:
# Dependencies
import os

# set spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True)
df.show()

In [ ]:
# Count the number of records (rows) in the dataset
df.count()

In [ ]:
# Create review_id_table dataframe
ebook_review_id_table_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
ebook_review_id_table_df.show()

In [ ]:
# Create review_id_table dataframe
ebook_review_id_table_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
ebook_review_id_table_df.show()

In [ ]:
# Create products_table dataframe
ebook_products_table_df = df.select(["product_id", "product_title"])
ebook_products_table_df.show()

In [ ]:
# Create customers_table dataframe
ebook_customers_table_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
ebook_customers_table_df.show()

In [ ]:
# Create vine_table dataframe
ebook_vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
ebook_vine_table_df.show()

In [ ]:
# Postgres setup
# Configure settings for RDS
mode = "append"
jdbc_url = "jdbc:postgresql://amazonebookreviews.c0xjlfeovn0x.us-west-2.rds.amazonaws.com:5432/amazonebookreviews"
config = {"user":"<postgres username>", 
          "password":"<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataframe to review_id_table in RDS
ebook_review_id_table_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to products_table in RDS
ebook_products_table_df.write.jdbc(url=jdbc_url, table="products_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to customers_table in RDS
ebook_customers_table_df.write.jdbc(url=jdbc_url, table="customers_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to vine_table in RDS
ebook_vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)